In [ ]:
from subprocess import Popen, PIPE, CalledProcessError
import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import platform
import fileinput
import os.path
import shutil
import getpass
from os import system
import time


options = {'stdout': PIPE, 'stderr': PIPE, 'bufsize' : 1, 'universal_newlines' : True, 'shell' : False}
if (platform.system() == 'Windows'):
    options['shell'] = True
    #Packages required to generate ssh keys in windows
    from cryptography.hazmat.primitives import serialization as crypto_serialization
    from cryptography.hazmat.primitives.asymmetric import rsa
    from cryptography.hazmat.backends import default_backend as crypto_default_backend

def callPopen(cmd):
    print(cmd)
    with Popen(cmd.split(),**options) as p:
        for line in p.stdout:
            print(line, end='')
        for line in p.stderr:
            print(line, end='')
        if p.returncode != (0 or None):
            raise CalledProcessError(p.returncode, p.args)

def isInstance(name):
    instanceExists=False
    ip=''
    with Popen('gcloud compute instances list'.split(),**options) as p:
        for line in p.stdout:
            if re.match('^{}'.format(name), line):
                instanceExists=True
                ip = line.strip().split()
                ip = ip[4]
        for line in p.stderr:
            print(line, end='')
        if p.returncode != (0 or None):
            raise CalledProcessError(p.returncode, p.args)
        return(instanceExists,ip)
    
            
def text_prepender(filename, text):
    with open(filename, 'r+') as f:
        content = f.read()
        f.seek(0, 0)
        f.write(text.rstrip('\r\n') + content)
        
def replace(file,pattern,replace):
    fileinput.close()
    for line in fileinput.input(file, inplace=True):
        print( re.sub(pattern,
                      replace,
                      line.rstrip()
                      ) 
             )

if (platform.system() == 'Windows'):
    def generateSSHKey(username,savePath):
        key = rsa.generate_private_key(
            backend=crypto_default_backend(),
            public_exponent=65537,
            key_size=2048
            )
        private_key = key.private_bytes(
            crypto_serialization.Encoding.PEM,
            crypto_serialization.PrivateFormat.TraditionalOpenSSL,
            crypto_serialization.NoEncryption()
            )
        public_key = key.public_key().public_bytes(
            crypto_serialization.Encoding.OpenSSH,
            crypto_serialization.PublicFormat.OpenSSH
            )
        public_file = os.path.join(savePath,username + '.pub')
        private_file = os.path.join(savePath,username)
        text_file = open(public_file, "w")
        text_file.write(public_key.decode('utf-8') + ' ' + username)
        text_file.close()
        text_file = open(private_file, "w")
        text_file.write(private_key.decode('utf-8'))
        text_file.close()
        print('Successfully created key pair')
            
if (platform.system() == 'Linux'):
    def generateSSHKey(username,savePath):
        p = Popen("echo 'yes' | ssh-keygen -t rsa -f {0}/{1} -C {1} -N '' ".format(savePath,username),
              stdout=PIPE,
              shell=True,
              stderr=PIPE
               )
        print(p.communicate())    
        
#Asking for a password for the vnc server
startViewer=True
if startViewer:
    password = getpass.getpass('Password for VNC server? ')

# Automated compilation of the INCA core and its dependencies

## Pre-requisites

* gcloud command line utility
* google cloud account, you might need to be an admin for some of this to work, also this is not free
* ssh key setup in gitlab.au.dk
* fabric3 needs to be (pip) installed
* python modules listed in the first cell of this notebook
* In order to avoid repeatedly creating ssh keys to log on to git up, mirror the INCA repository to google cloud and clone from there using gcloud. Do not forget to install git in the virtual machine as well as setting the necessary scopes (see below) for it to work.

## Creating an instance on Google Cloud

In [ ]:
#Variables to set
instanceName = 'core-base'
username = 'jose-luis'
keyDir = ('/home/jose-luis/.ssh/coreKeys')  #Dir where the ssh key to the instance will be stored

In [ ]:
createInstance = '''\
gcloud compute instances create {} \
--zone europe-west3-a \
--image-family ubuntu-1604-lts \
--image-project ubuntu-os-cloud \
--machine-type n1-standard-2 \
--scopes=default,cloud-platform,https://www.googleapis.com/auth/source.full_control
'''
#--boot-disk-size 200GB \
#--image-family ubuntu-1604-lts \ debian-9
#--image-project ubuntu-os-cloud \ debian-cloud
#gcloud compute instance-templates create

deleteInstance = '''\
gcloud compute instances delete {} \
--zone europe-west-3a \
'''

listInstances = '''gcloud compute instances list'''

addSSHKeys = '''gcloud compute instances add-metadata {} --zone europe-west3-a --metadata-from-file ssh-keys={}'''
addFirewallRule = '''gcloud compute instances add-tags {} --zone europe-west3-a --tags vnc-server'''

ip=''
instanceExists,ip = isInstance(instanceName)

if (ip != ''):
    print('Instance {} is {}'.format(instanceName,ip) )

isStarted = False
if instanceExists and ip == 'TERMINATED' :
    callPopen('gcloud compute instances start {} --zone europe-west3-a'.format(instanceName))
    instanceExists,ip = isInstance(instanceName)
    isStarted = True
    print("Machine started and ip is {}".format(ip))

wasCreated=False
if not instanceExists and not isStarted:
    callPopen(createInstance.format(instanceName))
    wasCreated=True
    if os.path.exists(keyDir):
        shutil.rmtree(keyDir)
    os.mkdir(keyDir)
    generateSSHKey(username,keyDir)
    keyFile = os.path.join(keyDir,username + '.pub')
    text_prepender('{}/{}.pub'.format(keyDir,username), '{}:'.format(username) )
    callPopen(addSSHKeys.format(instanceName,keyDir + '/{}.pub'.format(username)))
    callPopen(addFirewallRule.format(instanceName))
    #callPopen('sed -i s/^{0}:// {1}/{0}.pub'.format(username,keyDir))
    replace(keyFile,"^{}:".format(username),"")
    ip=isInstance(instanceName)[1]
    #callPopen('chmod 600 {}'.format(keyDir +'/' + username))
        
print("The ip of {} is {}".format(instanceName,ip))

## Getting link to download latest boost  and sqlite3 releases

In [ ]:
#boost
r = requests.get('https://www.boost.org/users/download/')
soup = BeautifulSoup(r.text)
table = soup.find( "table",{'class': 'download-table'})
data=[]
for link in table.find_all('a'):
    data.append(link['href'])
boostLink=[s for s in data if 'tar.gz' in s][0]
print(boostLink)
boostVersion=os.path.basename(boostLink).split('.')[0]

#sqlite3
#The webpage is rendered using JS so additional tricks are required
#You need to download chromedriver and put it in a directory that's in the search path
url='https://www.sqlite.org/download.html'
opt = webdriver.ChromeOptions()
opt.binary_location = '/usr/bin/google-chrome' #Point to the right location
opt.add_argument('headless')
browser = webdriver.Chrome(executable_path='./chromedriver',chrome_options=opt)
browser.get(url)
innerHTML = browser.execute_script("return document.body.innerHTML")
#print(innerHTML)
soup = BeautifulSoup(innerHTML)
table = soup.find( "table")
table_body = table.find('tbody')
data=[]
for link in soup.find_all('a',id=True):
    data.append(link['href'])
sqliteLink='https://www.sqlite.org/' + [s for s in data if 'autoconf' in s][0]
print(sqliteLink)

## Updating fabfile.py with credentials and ip

In [ ]:
callPopen("sed -i s/^env\.hosts.*/env.hosts=\['{}']/ fabfile.py".format(ip))
callPopen("sed -i s/^env\.user.*/env.user=\'{}\'/ fabfile.py".format(username))
callPopen("sed -i s$^env\.key_filename.*$env\.key_filename='{}'$ fabfile.py".format(keyDir + '/' + username))
callPopen("sed -i s/^env\.roledefs.*/env.roledefs={{\\'{}\\':[\\'{}\\'],/ fabfile.py".format('stage',ip))

## Testing connection

In [ ]:
time.sleep(5) #sometimes the fabfile editing takes too long and the connection fails
#we use sleep to avoid that. Alternatively, we should use subprocess.wait() in the above cell
# and modify Popen

#Testing connection
#Adding key to remote machine
callPopen("ssh -i {0}/{1} {1}@{2} -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no".format(keyDir,username,ip))
#p = Popen("ssh -i {0}/{1} {1}@{2} -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no".format(keyDir,username,ip),shell=True,stdout=PIPE,stdin=PIPE)
#p.wait()
#print(p.communicate())

## Editing core makefile to point to the right directories

In [ ]:
file="./Makefile"
replace(file, "^CND\\_PLATFORM=.*", "CND_PLATFORM={}".format("Linux"))
replace(file, "^BASEDIR=.*",        "BASEDIR={}".format("/home/" + username))
replace(file, "^BOOSTDIR=.*",       "BOOSTDIR=$(BASEDIR)/{}".format(boostVersion))
replace(file, "^TINYXMLDIR=.*",     "TINYXMLDIR={}".format("$(BASEDIR)/tinyxml2"))
replace(file, "^INCASRC=.*",        "INCASRC={}".format("${BASEDIR}/INCA/src"))
replace(file, "^INCAINC=.*",        "INCAINC={}".format("${BASEDIR}/INCA/include"))
replace(file, "^SQLITEDIR=.*",      "SQLITEDIR={}".format("$(BASEDIR)/sqlite"))
replace(file, "^PREFIX=.*",         "PREFIX={}".format("/home/" + username + "/local"))

## Editing libraries makefile to point to the right directories

In [ ]:
file="./Makelib"
replace(file, "^CND\\_PLATFORM=.*", "CND_PLATFORM={}".format("Linux"))
replace(file, "^BASEDIR=.*",        "BASEDIR={}".format("/home/" + username))
replace(file, "^BOOSTDIR=.*",       "BOOSTDIR=$(BASEDIR)/{}".format(boostVersion))
replace(file, "^TINYXMLDIR=.*",     "TINYXMLDIR={}".format("$(BASEDIR)/tinyxml2"))
replace(file, "^INCASRC=.*",        "INCASRC={}".format("${BASEDIR}/INCA/src"))
replace(file, "^INCAINC=.*",        "INCAINC={}".format("${BASEDIR}/INCA/include"))
replace(file, "^SQLITEDIR=.*",      "SQLITEDIR={}".format("$(BASEDIR)/sqlite"))
replace(file, "^PREFIX=.*",         "PREFIX={}".format("/home/" + username + "/local"))

## Editing tests makefile to point to the right directories

In [ ]:
file="./Maketests"
replace(file, "^CND\\_PLATFORM=.*", "CND_PLATFORM={}".format("Linux"))
replace(file, "^BASEDIR=.*",        "BASEDIR={}".format("/home/" + username))
replace(file, "^BOOSTDIR=.*",       "BOOSTDIR=$(BASEDIR)/{}".format(boostVersion))
replace(file, "^TINYXMLDIR=.*",     "TINYXMLDIR={}".format("$(BASEDIR)/tinyxml2"))
replace(file, "^INCASRC=.*",        "INCASRC={}".format("${BASEDIR}/INCA/src"))
replace(file, "^INCAINC=.*",        "INCAINC={}".format("${BASEDIR}/INCA/include"))
replace(file, "^SQLITEDIR=.*",      "SQLITEDIR={}".format("$(BASEDIR)/sqlite"))
replace(file, "^PREFIX=.*",         "PREFIX={}".format("/home/" + username + "/local"))

## Setting up the INCA core in the remote machine

In [ ]:
time.sleep(5) #to allow the file edits to complete
callPopen('fab testConnection')
callPopen("fab getUtilities")
if startViewer:
    callPopen("fab getGnomeAndVNC:'{}'".format(password))
callPopen("fab downloadSources:'{}','{}'".format(boostLink,sqliteLink))
callPopen('fab compileModels')
callPopen('fab examples')
callPopen('fab tests')
callPopen('fab setEnv')

In [ ]:
#Starting vncserver on remote machine
if startViewer:
    callPopen("fab startVNCServer")
# Creating an auto-closing tunnel for vncviewer and starting it
#Haven't figured out the syntax for subprocess.Popen and using os.system instead
#callPopen( "ssh -i {0}/{1} -f -L 5901:127.0.0.1:5901 -l {1} {2} sleep 10; vncviewer localhost:5901 &".
#format(keyDir,username,ip))
    system("ssh -f -i {0}/{1} -L 5901:127.0.0.1:5901 -l {1} {2} sleep 10; vncviewer localhost:5901 &".format(keyDir,username,ip))

## Setting up instance as a template

In [ ]:
createTemplate=False
cmd='''gcloud compute instance-templates create core-template \
    --source-instance={} \
    --source-instance-zone=europe-west3-a \
 '''.format(instanceName)

if createTemplate:
    callPopen(cmd)